In [1]:
import os
os.environ['CUPY_ACCELERATORS'] = 'cutensor'
import numpy as np
import cupy as cp
from scipy import fft
import zarr
from matplotlib import pyplot, colors, cm
from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm, trange

mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

def bytesize_string(nbytes):
    unit =          ['B',  'KiB' ,  'MiB' ,  'GiB' ,  'TiB' ,  'PiB' ,  'EiB' ,  'ZiB' ,  'YiB' ]
    size = np.array([ 1 , 1/2**10, 1/2**20, 1/2**30, 1/2**40, 1/2**50, 1/2**60, 1/2**70, 1/2**80]) * nbytes
    order_of_magnitude = np.argmax(size < 1.0) - 1
    return '{} {}'.format(size[order_of_magnitude], unit[order_of_magnitude])

def array_stats(a):
    print('{} × {} = {} | min: {}, max: {}, avg: {}'.format(a.shape, a.dtype, bytesize_string(a.nbytes), np.amin(a), np.amax(a), np.average(a)))

def display_image(values, color_map=None):
    values = values if color_map is None else cm.get_cmap(color_map)(values)
    display(Image.fromarray((np.flip(np.swapaxes(values, 0, 1), axis=0) * 255.0).astype(np.uint8)))
    
def save_image(name, values, color_map=None):
    values = values if color_map is None else cm.get_cmap(color_map)(values)
    Image.fromarray((np.flip(np.swapaxes(values, 0, 1), axis=0) * 255.0).astype(np.uint8)).save('figures/{}.png'.format(name))

In [2]:
# Focus range
n_f = 100
zeta_F = np.linspace(0.0, 5.0, num=n_f) / 1000

In [3]:
n_r = 1024

n_u = 20
n_v = 1080
chunk_v = 40
n_chunk = n_v//chunk_v

n_res = 4

n_a = np.arange(1, n_res+1, dtype=np.uint32) * 12
n_b = np.arange(1, n_res+1, dtype=np.uint32) * 140

In [4]:
projection = []

for index_res in range(n_res):
    projection.append(zarr.open('data/matrices/projection-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r'))

n_ab = [l.shape[1] for l in projection]

In [5]:
# Number of light field groups.
chunk_group = 2
n_group = n_chunk - (chunk_group - 1)

print('\nLight field group size:', chunk_group)
print('Number of light field groups:', n_group)


Light field group size: 2
Number of light field groups: 26


In [6]:
%%time

scenes = ['car', 'chess', 'dragon', 'sponza']
n_scenes = len(scenes)

projection_coef = [np.zeros((n_ab[index_res], n_ab[index_res], 3), dtype=np.float32) for index_res in range(n_res)]

n_iter = 50
rng = cp.random.default_rng()

mse_discrete = np.zeros((n_res, n_f), dtype=np.float32)

for target_index in np.arange(69, 100, dtype=np.uint8):
    #with tqdm(total=n_scenes * n_res * (n_group * n_group + n_iter + n_f)) as pbar:
    with tqdm(total=n_group * n_group + n_iter + n_f) as pbar:
        element_indices_per_group = []
        for index_res in range(n_res):
            chunk_incidence_per_element = np.any(projection[index_res][target_index], axis=(1, 3))
            element_assignment_per_group = np.minimum(np.argmax(chunk_incidence_per_element, axis=1), n_group-1).reshape(1, n_ab[index_res]) == np.arange(n_group).reshape(n_group, 1)
            element_indices_per_group.append([np.nonzero(element_assignment)[0] for element_assignment in element_assignment_per_group])
        
        #for scene in scenes:
        for scene in ['sponza']:
            print(scene, target_index)
            # Load scene.
            light_field = zarr.open('data/' + scene + '_sampled.zarr', mode='r')

            # Load initial values from sampled light field.
            sample_values = np.empty((n_u, chunk_group, chunk_v, n_u, n_chunk, chunk_v, 3), dtype=np.float32)
            sample_values[:, 1:, :] = light_field[:, :chunk_group-1, :]

            for idx_group in range(n_group):
                slice_x = slice(idx_group, idx_group + chunk_group)
                
                # Load more values from sampled light field.
                sample_values[:, :-1, :] = sample_values[:, 1:, :]
                sample_values[:, -1, :] = light_field[:, idx_group+chunk_group-1, :]

                for idy_group in range(n_group):
                    slice_y = slice(idy_group, idy_group + chunk_group)

                    # Load sample values into the GPU for this group.
                    sample_values_GPU = cp.array(sample_values[:, :, :, :, slice_y, :].reshape(n_u * chunk_group * chunk_v, n_u * chunk_group * chunk_v, 3))

                    #for index_res in range(n_res):
                    for index_res in [3]:
                        # Display elements assigned to this group.
                        idx_element = element_indices_per_group[index_res][idx_group]
                        idy_element = element_indices_per_group[index_res][idy_group]

                        # Compute discrete projection.
                        lm_x_GPU = cp.array(projection[index_res].oindex[target_index, idx_element, :, slice_x, :].reshape(-1, n_u * chunk_group * chunk_v))
                        lm_y_GPU = cp.array(projection[index_res].oindex[target_index, idy_element, :, slice_y, :].reshape(-1, n_u * chunk_group * chunk_v))
                        projection_coef[index_res][np.ix_(idx_element, idy_element)] = cp.einsum('hx,vy,xyc->hvc', lm_x_GPU, lm_y_GPU, sample_values_GPU).get()

                        # Update progress bar.
                        pbar.update()

                    # Free GPU memory
                    del sample_values_GPU
                    del lm_x_GPU
                    del lm_y_GPU
                    mempool.free_all_blocks()
                        
            reference_images = zarr.open('data/{}/reference.zarr'.format(scene), mode='r')

            #for index_res in range(n_res):
            for index_res in [3]:
                simulation = cp.array(zarr.open('data/matrices/simulation-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r'))

                # Discrete coefficients.
                coefficients = 1.0 - rng.random((n_ab[index_res], n_ab[index_res], 3), dtype=cp.float32)
                proj_GPU = cp.array(projection_coef[index_res])
                autocorrelation = cp.array(zarr.open('data/matrices/autocorrelation-{}x{}.zarr'.format(n_a[index_res], n_b[index_res]), mode='r')[target_index])
                # For each multiplicative rule iteration...
                for iteration in range(n_iter):
                    # Apply the autocorrelation linear map on current coefficients.
                    temp = cp.tensordot(autocorrelation, cp.tensordot(autocorrelation, coefficients, axes=(1, 1)), axes=(1, 1))
                    # Divide the projected coefficients by the autocorrelated coefficients.
                    temp = cp.add(temp, cp.finfo(cp.float32).eps, out=temp)
                    temp = cp.divide(proj_GPU, temp, out=temp)
                    # Update current coefficients with the multiplicative rule.
                    coefficients = cp.multiply(coefficients, temp, out=coefficients)
                    # Clip coefficients to interval [0, 1].
                    coefficients = cp.clip(coefficients, 0.0, 1.0, out=coefficients)

                    # Update progress bar.
                    pbar.update()

                # Discrete retinal images.
                for index_f in range(n_f):
                    retinal_image = cp.tensordot(simulation[index_f], cp.tensordot(simulation[index_f], coefficients, axes=(1, 1)), axes=(1, 1)).get()
                    mse_discrete[index_res, index_f] = np.mean(np.square(reference_images[index_f] - retinal_image))
                    if index_f == target_index:
                        save_image('{}/{}x{}/f{}-discrete'.format(scene, n_a[index_res], n_b[index_res], index_f), retinal_image)
                    
                    # Update progress bar.
                    pbar.update()

                # Free GPU memory
                del simulation
                del coefficients
                del proj_GPU
                del autocorrelation
                del temp
                mempool.free_all_blocks()

                #zarr.open('data/{}/mse-discrete.zarr'.format(scene), mode='a', shape=(n_f, n_res, n_f), chunks=(1, n_res, n_f), dtype=np.float32)[target_index] = mse_discrete
                zarr.open('data/{}/mse-discrete.zarr'.format(scene), mode='a', shape=(n_f, n_res, n_f), chunks=(1, n_res, n_f), dtype=np.float32)[target_index, 3] = mse_discrete[3]

  0%|          | 0/826 [00:00<?, ?it/s]

sponza 69


  0%|          | 0/826 [00:00<?, ?it/s]

sponza 70


  0%|          | 0/826 [00:00<?, ?it/s]

...

sponza 97


  0%|          | 0/826 [00:00<?, ?it/s]

sponza 98


  0%|          | 0/826 [00:00<?, ?it/s]

sponza 99
Wall time: 9h 1min 48s
